In [2]:
import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'

from jax.config import config
config.update("jax_enable_x64", True)
import jax
from jax import jit
import jax.numpy as jnp

from polynomials.pr_func import make_polynomial_ring_fn
from ckks.rlwe_jit import Encryptor
from ckks.utils import ring_polymul, get_modulo, shift_mod

In [73]:
n = 8
q = 3
P = 1023
seed = 5
std = 1
p = 3
L = 15
h = 2

In [74]:
rlwe = Encryptor(n, q, p, L, P, std, h, seed)

# rlwe.Q = 671082899

sk, pk = rlwe.generate_keys()

m1 = jnp.arange(111,889, 111)  # plaintext
m2 = jnp.flip(m1)

## Add & Mul

In [75]:
pk

(DeviceArray([ -8831877.,    921949., -10416868., -17795335.,   2994273.,
                3770557.,  11112864.,  12504424.], dtype=float64),
 DeviceArray([ 16559429, -15374430,  -2870006,   5961871,   5039922,
               15456789,  -9794597, -12788872], dtype=int64))

In [76]:
c1 = rlwe.encrypt(m1, pk)
c2 = rlwe.encrypt(m2, pk)

In [77]:
rlwe.decrypt(c1, L, sk)

DeviceArray([112., 221., 334., 450., 551., 665., 779., 892.], dtype=float64)

In [78]:
clow = rlwe.rescale(c1, L, L-2)

In [79]:
rlwe.decrypt(clow, L-2, sk)

DeviceArray([13., 24., 38., 50., 61., 74., 87., 99.], dtype=float64)

In [80]:
m1 + m2

DeviceArray([999, 999, 999, 999, 999, 999, 999, 999], dtype=int64)

In [81]:
rlwe.decrypt(
    rlwe.add(c1, c2, L),
    L, sk
)

DeviceArray([1001.,  997., 1001., 1011.,  991.,  997., 1003., 1007.], dtype=float64)

In [82]:
c_mul = rlwe.mul(c1, c2, L)
c_mul

(DeviceArray([ 21474134.,   -766011.,  14921263., -20781493.,   7077716.,
               -3491477.,     34154.,  21353610.], dtype=float64),
 DeviceArray([ -5998675.,  -4615894.,  18659763.,   -340121.,   2429467.,
              -11173830.,  10349478.,   3237227.], dtype=float64))

In [83]:
c1

(DeviceArray([-14946616., -18522494.,  -8976073.,  16950912.,   7812471.,
                8136547.,   4369741.,  -1408394.], dtype=float64),
 DeviceArray([ -6253631.,  -1884669.,  -3293955.,  11652773., -12871233.,
               -3894826., -20845288.,  14389513.], dtype=float64))

In [84]:
rlwe.decrypt(c_mul, L, sk)

DeviceArray([ 2474250.,  2003577.,  1317966.,   741815.,   -34535.,
              -730029., -1348627., -1922047.], dtype=float64)

In [1]:
ring_polymul(m1, m2, get_modulo(8))[1][-8:]

NameError: name 'ring_polymul' is not defined

In [93]:
jnp.abs(rlwe.decrypt(c_mul, L, sk) - ring_polymul(m1, m2, get_modulo(8))[1][-8:]).sum() / (ring_polymul(m1, m2, get_modulo(8))[1][-8:]).sum()

DeviceArray(0.11586229, dtype=float64)